In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!nvidia-smi -L || echo "No GPU"


Mounted at /content/drive
GPU 0: Tesla T4 (UUID: GPU-b7367d9f-d30f-6735-4966-508cb02d1361)


In [5]:
import os, zipfile, glob

zip_path     = "/content/drive/MyDrive/DIV2K_519sampled.zip"
extract_root = "/content/drive/MyDrive/DIV2K_519sampled"

os.makedirs(extract_root, exist_ok=True)
if zipfile.is_zipfile(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_root)

print("압축 해제 위치:", extract_root)
print("상위 폴더 목록:", os.listdir(extract_root)[:10])


압축 해제 위치: /content/drive/MyDrive/DIV2K_519sampled
상위 폴더 목록: ['DIV2K_519sampled']


In [6]:
import os, glob

def find_real_root(root):
    IMG_EXT = ('*.png','*.jpg','*.jpeg','*.bmp','*.webp','*.tif','*.tiff')
    files = []
    for ext in IMG_EXT:
        files += glob.glob(os.path.join(root, ext))
        files += glob.glob(os.path.join(root, "**", ext), recursive=True)
    if files:
        return root

    subs = [os.path.join(root, d) for d in os.listdir(root)
            if os.path.isdir(os.path.join(root, d))]
    if len(subs) == 1:
        return find_real_root(subs[0])
    return root

real_root = find_real_root(extract_root)
print("이미지 실제 루트:", real_root)

IMG_EXT = ('*.png','*.jpg','*.jpeg','*.bmp','*.webp','*.tif','*.tiff')
all_images = []
for ext in IMG_EXT:
    all_images += glob.glob(os.path.join(real_root, ext))
    all_images += glob.glob(os.path.join(real_root, "**", ext), recursive=True)

all_images = sorted(set(all_images))
print("총 이미지 수:", len(all_images))
print("샘플:", all_images[:3])


이미지 실제 루트: /content/drive/MyDrive/DIV2K_519sampled
총 이미지 수: 519
샘플: ['/content/drive/MyDrive/DIV2K_519sampled/DIV2K_519sampled/0001.png', '/content/drive/MyDrive/DIV2K_519sampled/DIV2K_519sampled/0002.png', '/content/drive/MyDrive/DIV2K_519sampled/DIV2K_519sampled/0003.png']


In [7]:
import random
random.seed(0)

random.shuffle(all_images)
n = len(all_images)
n_train = max(1, int(n*0.7))
n_valid = max(1, int(n*0.2))
n_test  = max(1, n - n_train - n_valid)

train_files = all_images[:n_train]
valid_files = all_images[n_train:n_train+n_valid]
test_files  = all_images[n_train+n_valid:]

print(f"split -> train:{len(train_files)}, valid:{len(valid_files)}, test:{len(test_files)}")


split -> train:363, valid:103, test:53


In [8]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch

#해상도
crop_size = 128
upscale   = 2

transform_hr = transforms.Compose([
    transforms.CenterCrop(crop_size),
    transforms.ToTensor()
])

transform_lr = transforms.Compose([
    transforms.CenterCrop(crop_size),
    transforms.Resize(crop_size // upscale, interpolation=Image.BICUBIC),
    transforms.Resize(crop_size, interpolation=Image.BICUBIC),
    transforms.ToTensor()
])

class SRDataset(Dataset):
    def __init__(self, files, t_hr, t_lr):
        self.files = files
        self.t_hr = t_hr
        self.t_lr = t_lr
    def __len__(self):
        return len(self.files)
    def __getitem__(self, idx):
        img = Image.open(self.files[idx]).convert("RGB")
        hr  = self.t_hr(img)
        lr  = self.t_lr(img)
        return lr, hr

train_dataset = SRDataset(train_files, transform_hr, transform_lr)
valid_dataset = SRDataset(valid_files, transform_hr, transform_lr)
test_dataset  = SRDataset(test_files,  transform_hr, transform_lr)

def _safe_bs(n, target=16):
    if n == 0:
        raise ValueError("Dataset 비어있음: 이미지 수집/경로/확장자를 다시 확인하세요.")
    return min(target, n)

train_loader = DataLoader(train_dataset, batch_size=_safe_bs(len(train_dataset), 16),
                          shuffle=True, num_workers=2, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=_safe_bs(len(valid_dataset), 16),
                          shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=1,
                          shuffle=False, num_workers=2, pin_memory=True)

print("lens:", len(train_dataset), len(valid_dataset), len(test_dataset))
xb, yb = next(iter(train_loader))
print("lr:", xb.shape, "hr:", yb.shape)


lens: 363 103 53
lr: torch.Size([16, 3, 128, 128]) hr: torch.Size([16, 3, 128, 128])


In [9]:
import torch.nn as nn
import torch

class SRCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu  = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)
        self.conv3 = nn.Conv2d(32, 3,  kernel_size=5, padding=2)
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model  = SRCNN().to(device)
print("device:", device)


device: cuda


In [10]:
import torch.optim as optim
import torch.nn.functional as F

criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 50
best_val   = float('inf')
ckpt_path  = "/content/drive/MyDrive/srcnn_best.pth"

for epoch in range(1, num_epochs+1):
    model.train()
    run_loss = 0.0
    for lr_imgs, hr_imgs in train_loader:
        lr_imgs = lr_imgs.to(device, non_blocking=True)
        hr_imgs = hr_imgs.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        sr = model(lr_imgs)
        loss = criterion(sr, hr_imgs)
        loss.backward()
        optimizer.step()
        run_loss += loss.item() * lr_imgs.size(0)

    train_loss = run_loss / len(train_loader.dataset)

    model.eval()
    val_run = 0.0
    with torch.no_grad():
        for lr_imgs, hr_imgs in valid_loader:
            lr_imgs = lr_imgs.to(device, non_blocking=True)
            hr_imgs = hr_imgs.to(device, non_blocking=True)
            sr = model(lr_imgs)
            vloss = criterion(sr, hr_imgs)
            val_run += vloss.item() * lr_imgs.size(0)
    val_loss = val_run / len(valid_loader.dataset)

    print(f"[{epoch:03d}/{num_epochs}] train: {train_loss:.6f} | valid: {val_loss:.6f}")

    if val_loss < best_val:
        best_val = val_loss
        torch.save({"model": model.state_dict()}, ckpt_path)
        print(f" ↳ best saved: {ckpt_path} (val={best_val:.6f})")


[001/50] train: 0.285179 | valid: 0.159030
 ↳ best saved: /content/drive/MyDrive/srcnn_best.pth (val=0.159030)
[002/50] train: 0.113636 | valid: 0.101443
 ↳ best saved: /content/drive/MyDrive/srcnn_best.pth (val=0.101443)
[003/50] train: 0.089978 | valid: 0.082863
 ↳ best saved: /content/drive/MyDrive/srcnn_best.pth (val=0.082863)
[004/50] train: 0.075836 | valid: 0.070304
 ↳ best saved: /content/drive/MyDrive/srcnn_best.pth (val=0.070304)
[005/50] train: 0.068455 | valid: 0.066837
 ↳ best saved: /content/drive/MyDrive/srcnn_best.pth (val=0.066837)
[006/50] train: 0.065506 | valid: 0.063731
 ↳ best saved: /content/drive/MyDrive/srcnn_best.pth (val=0.063731)
[007/50] train: 0.062809 | valid: 0.060787
 ↳ best saved: /content/drive/MyDrive/srcnn_best.pth (val=0.060787)
[008/50] train: 0.059993 | valid: 0.058013
 ↳ best saved: /content/drive/MyDrive/srcnn_best.pth (val=0.058013)
[009/50] train: 0.056766 | valid: 0.055334
 ↳ best saved: /content/drive/MyDrive/srcnn_best.pth (val=0.055334)
[

In [14]:
import os, glob, torch, torchvision
from PIL import Image
from torchvision import transforms as T

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device).eval()

try:
    _ = all_images
except NameError:
    IMG_EXT = ('.png','.jpg','.jpeg','.bmp','.webp','.tif','.tiff')
    all_images = []
    for root, _, files in os.walk(real_root):
        for f in files:
            if os.path.splitext(f)[1].lower() in IMG_EXT:
                all_images.append(os.path.join(root, f))
    all_images = sorted(set(all_images))

print("총 처리 대상:", len(all_images), "장")

out_dir = "/content/drive/MyDrive/SR_outputs_x2"
os.makedirs(out_dir, exist_ok=True)
to_tensor = T.ToTensor()

scale = 2

saved = 0
model.eval()
with torch.no_grad():
    for path in all_images:
        img = Image.open(path).convert("RGB")
        w, h = img.size
        up_img = img.resize((w*scale, h*scale), resample=Image.BICUBIC)

        x = to_tensor(up_img).unsqueeze(0).to(device)
        sr = model(x).clamp(0, 1)

        base = os.path.splitext(os.path.basename(path))[0]
        out_path = os.path.join(out_dir, f"{base}_SRx2.png")
        torchvision.utils.save_image(sr, out_path)

        saved += 1
        if saved % 25 == 0:
            print(f"- 진행 상황: {saved}/{len(all_images)} 저장 완료")

print(f"[완료] {saved}장 저장 → {out_dir}")


총 처리 대상: 519 장
- 진행 상황: 25/519 저장 완료
- 진행 상황: 50/519 저장 완료
- 진행 상황: 75/519 저장 완료
- 진행 상황: 100/519 저장 완료
- 진행 상황: 125/519 저장 완료
- 진행 상황: 150/519 저장 완료
- 진행 상황: 175/519 저장 완료
- 진행 상황: 200/519 저장 완료
- 진행 상황: 225/519 저장 완료
- 진행 상황: 250/519 저장 완료
- 진행 상황: 275/519 저장 완료
- 진행 상황: 300/519 저장 완료
- 진행 상황: 325/519 저장 완료
- 진행 상황: 350/519 저장 완료
- 진행 상황: 375/519 저장 완료
- 진행 상황: 400/519 저장 완료
- 진행 상황: 425/519 저장 완료
- 진행 상황: 450/519 저장 완료
- 진행 상황: 475/519 저장 완료
- 진행 상황: 500/519 저장 완료
[완료] 519장 저장 → /content/drive/MyDrive/SR_outputs_x2
